# Twilio Gateway Management

## Setup

In [1]:
!pip install Faker ipython_secrets pandas twilio

In [2]:
from ipython_secrets import get_secret
TWILIO_ACCOUNT_SID = get_secret('TWILIO_ACCOUNT_SID', 'olin-build')
TWILIO_AUTH_TOKEN = get_secret('TWILIO_AUTH_TOKEN', 'olin-build')
MQTT_URL = get_secret('MQTT_URL', 'olin-build')

## Provisioning Phone Numbers

Create a Twilio client, and test it.

Read the class Roster.

In [3]:
from twilio.rest import Client

client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
numbers = client.available_phone_numbers("US") \
                .local \
                .list(area_code="617")

len(numbers)

30

In [4]:
import pandas as pd
df = pd.read_csv('data/HtL Roster.csv')

# df.head()  # <- FERPA

Scrub student names so this can be published on the open web. Skip this in actual use.

In [5]:
from faker import Faker

fake = Faker()
df['First Name'] = [fake.first_name() for _ in range(len(df))]
df['Last Name'] = [fake.last_name() for _ in range(len(df))]

df.head()

,First Name,Last Name,Email,GitHub Login,Twilio Phone Number
0,Shelly,Brown,NaN,NaN,NaN
1,Tracy,Jackson,NaN,NaN,NaN
2,Ashley,Oneal,NaN,NaN,NaN
3,Robert,Powell,NaN,NaN,NaN
4,Theresa,Bolton,NaN,NaN,NaN


Find some numbers.

In [6]:
available_numbers = client.available_phone_numbers("US") \
    .local \
    .list(area_code="617")

assert len(available_numbers) >= len(df), "Large class. Add a loop above."

Buy everyone a phone number.

In theory, `create` can fail because of a race between `list` and `create`.
In practice, this didn't happen, so this code doesn't handle it.

In [7]:
from itertools import starmap

def create_number(first_name, last_name, number):
    friendly_name=' '.join([first_name, last_name])
    return client.incoming_phone_numbers
        .create(phone_number=number.phone_number, friendly_name=friendly_name)

numbers = list(starmap(create_number, zip(df['First Name'], df['Last Name'], available_numbers)))
print('Provisioned', len(numbers), 'phone numbers:', ', '.join(number.phone_number for number in numbers))

Provisioned 13 phone numbers: +16179344108, +16178551060, +16179172199, +16173935534, +16179816315, +16176827845, +16175539529, +16172497676, +16173796027, +16177516532, +16179346307, +16173797614, +16178706551


In [8]:
df['Twilio SID'] = [number.sid for number in numbers]
df['Twilio Phone Number'] = [number.phone_number for number in numbers]
df.to_csv('data/HtL Phone Numbers.csv')
df.head()

,First Name,Last Name,Email,GitHub Login,Twilio Phone Number,Twilio SID
0,Shelly,Brown,NaN,NaN,+16179344108,PNb4e01d1a4a4469ceff6cbccced032ace
1,Tracy,Jackson,NaN,NaN,+16178551060,PN1f1dd8ea8656e9c632235148ab37db9f
2,Ashley,Oneal,NaN,NaN,+16179172199,PNe81eb2bc05880f0f610a3279c85bbb9c
3,Robert,Powell,NaN,NaN,+16173935534,PN5d014b7aaad2d177151578302455481c
4,Theresa,Bolton,NaN,NaN,+16179816315,PN5a492f3be08fe7872f4ce9d13f5fe652


Set the SMS gateway. For historical reasons, this is done in a separate step for the initial provisioning. Note the gateway name for this version open web is slightly different from what we're actually using.

In [9]:
len(number.update(sms_url='https://twilio-gateway-app.herokuapp.com/sms_webhook') for number in numbers)

13

Create and provision a single number.

In [10]:
number = create_number('Joe', 'Frank', client.available_phone_numbers("US").local.list(area_code="617"))
number.update(sms_url='https://twilio-gateway-app.herokuapp.com/sms_webhook')
number.sid, number.phone_number

## RabbitMQ Management

In my brief exploration, I didn't find a Python rabbit client that works with ssh, so I wrote this instead.

In [11]:
import subprocess
from itertools import starmap
from urllib.parse import urlparse

RABBITMQ_BIN_PATH = '/usr/local/Cellar/rabbitmq/3.7.2/sbin/rabbitmqadmin'

def rabbitmqadmin(*args, **kwargs):
    url = urlparse(MQTT_URL)
    admin_args = '-H termite.rmq.cloudamqp.com -P 443 --ssl -V djdnhvdd -u djdnhvdd -p PAevqYVXbfbUDzF0Xwe245jOAa37AzlF'.split()
    connection_args = {
        '-H': url.hostname,
        '-P': 443,  # use SSL instead of the url.port
        '-u': url.username,
        '-p': url.password,
        '-V': url.path.strip('/'),
    }
    admin_args = [str(x) for t in connection_args.items() for x in t] + ['--ssl']
    res = subprocess.run([RABBIT_BIN_PATH] + admin_args + list(args) + list(starmap('{}={}'.format, kwargs.items())),
                         stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    # This is an alternative to res.check_returncode(), that abuses the exception class to give a useful error message
    if res.returncode != 0:
        raise subprocess.CalledProcessError(res.returncode, res.stderr.decode().strip())
    return res.stdout.decode().strip()

print(rabbitmqadmin('list', 'exchanges'))

+--------------------+---------+
|        name        |  type   |
+--------------------+---------+
|                    | direct  |
| amq.direct         | direct  |
| amq.fanout         | fanout  |
| amq.headers        | headers |
| amq.match          | headers |
| amq.rabbitmq.trace | topic   |
| amq.topic          | topic   |
+--------------------+---------+


Create and bind the queues

In [12]:
for phone_number in df['Twilio Phone Number']:
    queue_name = phone_number.replace('+', 'incoming-sms-')
    print('creating and binding', queue_name)
    rabbitmqadmin('declare', 'queue', name=queue_name)
    rabbitmqadmin('declare', 'binding', source='amq.topic', destination_type='queue', destination=queue_name, routing_key=queue_name)

creating and binding incoming-sms-16179344108
creating and binding incoming-sms-16178551060
creating and binding incoming-sms-16179172199
creating and binding incoming-sms-16173935534
creating and binding incoming-sms-16179816315
creating and binding incoming-sms-16176827845
creating and binding incoming-sms-16175539529
creating and binding incoming-sms-16172497676
creating and binding incoming-sms-16173796027
creating and binding incoming-sms-16177516532
creating and binding incoming-sms-16179346307
creating and binding incoming-sms-16173797614
creating and binding incoming-sms-16178706551


Create and bind a queue for a single number

In [13]:
phone_number = '+16176741212'
queue_name = phone_number.replace('+', 'incoming-sms-')
print('creating and binding', queue_name)
rabbitmqadmin('declare', 'queue', name=queue_name)
rabbitmqadmin('declare', 'binding', source='amq.topic', destination_type='queue', destination=queue_name, routing_key=queue_name)

creating and binding incoming-sms-16176741212
